In [12]:
import io
import cv2
import base64
import requests


from PIL import Image


# A1111 URL
url = "http://10.8.76.222:7860"

# Read Image in RGB order
img = cv2.imread('test.png')

# Encode into PNG and send to ControlNet
retval, bytes = cv2.imencode('.png', img)
encoded_image = base64.b64encode(bytes).decode('utf-8')
# print(encoded_image)

# A1111 payload
payload = {
    "prompt": 'anime,face,girl,master piece',
    "negative_prompt": "",
    "batch_size": 1,
    "steps": 20,
    "cfg_scale": 7,
    "alwayson_scripts": {
        "controlnet": {
            "args": [
                {
                    "input_image": encoded_image,
                    "module": "openpose_faceonly",
                    "model": "control_v11p_sd15_openpose [cab727d4]",
                }
            ]
        }
    }
}


# Trigger Generation
response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

# Read results
r = response.json()
result = r['images'][0]
image = Image.open(io.BytesIO(base64.b64decode(result.split(",", 1)[0])))
image.save('output.png')

In [20]:
result = r["parameters"]["alwayson_scripts"]["controlnet"]["args"][0]["input_image"]
image = Image.open(io.BytesIO(base64.b64decode(result.split(",", 1)[0])))
image.save('input.png')
# r["parameters"]["alwayson_scripts"]["controlnet"]["args"]

In [24]:
r["parameters"]["alwayson_scripts"]["controlnet"]

{'args': [{'input_image': '...',
   'module': 'openpose_faceonly',
   'model': 'control_v11p_sd15_openpose [cab727d4]'}]}

In [9]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

#url = "http://127.0.0.1:7860"
url = "http://10.8.76.222:7860"

payload = {
    "prompt": "1girl,anime,face",
    "steps": 40
}

response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

r = response.json()
# print(r)
for i in r['images']:
    image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

    png_payload = {
        "image": "data:image/png;base64," + i
    }
    response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

    pnginfo = PngImagePlugin.PngInfo()
    pnginfo.add_text("parameters", response2.json().get("info"))
    image.save('output.png', pnginfo=pnginfo)

In [23]:
import requests
import random

f = {
    "prompt":{
        "default_prompt": "masterpiece, high quality, full body, a girl",
        "random_prompt":[
            "Red hair, blue hair, blond hair, silver hair",
            "School uniforms, Chinese dresses",
            "denim shorts,check skert,denim skert",
            "Long hair, short hair, twin-tail hair, ponytail",
            "city, Beach, countryside",
            "Spring, Summer, Autumn, Winter",
            "Cat ears, fox ears, bear ears"
        ]
    },
    "negative_prompt":{
        "default_negative_prompt": "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",
        "random_negative_prompt": [
            "dog year"
        ]
    },
    "params":{
        "restore_faces": "False",
        "face_restorer": "CodeFormer",
        "codeformer_weight": 0.5,
        "sd_model": "model.ckpt",
        "seed": -1,
        "seed_enable_extras": "False",
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": 0,
        "seed_resize_from_w": 0,
        "sampler_name": "Euler a",
        "steps": 30,
        "cfg_scale": 7.5,
        "denoising_strength": 0.35,
        "batch_count": 1,
        "batch_size": 1,
        "base_size": 512,
        "max_size": 768,
        "tiling": "False",
        "highres_fix": "False",
        "firstphase_height": 512,
        "firstphase_width": 512,
        "upscaler_name": "None",
        "filter_nsfw": "False",
        "include_grid": "False",
        "sample_path": "outputs/krita-out",
        "orig_width": 512,
        "orig_height": 512
    }
}

def bool_check(s):
    if isinstance( s, str ) and s.lower() == 'false':
        return False
    elif isinstance( s, str ) and s.lower() == 'true':
        return True
    return s

def main():
    # while True:
    for j in range(5):
        try:
            f['params']['prompt'] = f['prompt']['default_prompt'] + ''.join([' ,'+_.split(",")[random.randrange(len(_.split(",")))] for _ in f['prompt']['random_prompt']])
            f['params']['negative_prompt'] = f['negative_prompt']['default_negative_prompt'] + ''.join([' ,'+_.split(",")[random.randrange(len(_.split(",")))] for _ in f['negative_prompt']['random_negative_prompt']])

            for _ in f['params']:
                f['params'][_] = bool_check(f['params'][_])
            
            url = "http://10.8.76.222:7860"
            r = requests.post(url=f"{url}/sdapi/v1/txt2img", json=f['params']).json()
            print(r)
            # if r['outputs'] == []:
            #     print('rが空でした')
            #     break
			
            # print( 'Prompt：' + r['info'].split(':')[1].replace(', "all_prompts"','').replace('"','') + ' で生成しました。' )
            print("画像を保存します")
            for i in r['images']:
                image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))
                print("がぞうおおおお")
                png_payload = {
                    "image": "data:image/png;base64," + i
                }
                response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

                pnginfo = PngImagePlugin.PngInfo()
                pnginfo.add_text("parameters", response2.json().get("info"))
                image.save(f'output_{j}.png', pnginfo=pnginfo)
        except Exception as e:
            print(e)
            print(r)
            print('Error')
            break
        
if __name__ == "__main__":
    main()


{'images': ['...'], 'parameters': {'enable_hr': False, 'denoising_strength': 0.35, 'firstphase_width': 512, 'firstphase_height': 512, 'hr_scale': 2.0, 'hr_upscaler': None, 'hr_second_pass_steps': 0, 'hr_resize_x': 0, 'hr_resize_y': 0, 'hr_sampler_name': None, 'hr_prompt': '', 'hr_negative_prompt': '', 'prompt': 'masterpiece, high quality, full body, a girl , silver hair , Chinese dresses ,check skert ,Long hair ,city , Autumn , bear ears', 'styles': None, 'seed': -1, 'subseed': -1, 'subseed_strength': 0.0, 'seed_resize_from_h': 0, 'seed_resize_from_w': 0, 'sampler_name': 'Euler a', 'batch_size': 1, 'n_iter': 1, 'steps': 30, 'cfg_scale': 7.5, 'width': 512, 'height': 512, 'restore_faces': False, 'tiling': False, 'do_not_save_samples': False, 'do_not_save_grid': False, 'negative_prompt': 'lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry ,dog y